<a href="https://colab.research.google.com/github/lim-jr/ESAA/blob/main/%EC%8A%A4%ED%83%9C%ED%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10 스태킹 앙상블

## 1. 스태킹 앙상블 개요

* 스태킹은 개별적인 여러 알고리즘을 서로 결합해 예측결과를 도출한다. (배깅, 부스팅 공통점)
* 개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측을 수행한다. (배깅, 부스팅 차이점)
* 즉, 개별 알고리즘의 예측결과 데이터세트를/ 최종적인 메타 데이터세트로 만들어/ 별도의 ML알고리즘으로 최종학습을 수행하고//   
테스트 데이터를 기반으로/ 다시 최종예측을 수행하는 방식


* 스태킹 모델은 두 종류의 모델이 필요하다  
  1) 첫번째는 개별적인 기반 모델  
  2) 두번째는 이 개별 기반 모델의 예측 데이터를/ 학습 데이터로 만들어서 학습하는 최종 메타 모델  
*  스태킹 모델의 핵심은,   
  1) 개별 모델의 예측 데이터를 각각 스태킹 형태로 결합해   
  2) 최종 메타 모델의 학습용 피처 데이터셋과 테스트용 피처 데이터셋을 만드는 것이다.   
  (교과서 다이어그램 참조 p.279)
* 스태킹을 현실모델에 적용하는 경우는 많지 않지만, 대회에서 조금이라도 성능 수치를 높여야 할 경우 자주 사용됨.  
  반드시 성능향상이 보장되는 것은 아니지만, 성능이 비슷한 모델을 결합해 좀 더 나은 성능향상을 도출하기 위해 적용 


## 2. 기본 스태킹 모델

In [ ]:
import numpy as np

# 1) 개별 모델은 KNN, 랜덤포레스트, 에이다부스트, 결정트리이며
# 2) 최종메타 모델은 개별모델의 예측 결과를 합한 데이터 세트로 학습/예측을 하는 로지스틱 회귀
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

### Step1) 모델 객체 생성

In [ ]:
# 1) 개별 ML 모델 생성 
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 2) 스태킹으로 만들어진 데이터세트를 학습, 예측할 최종모델
lr_final = LogisticRegression(C=10)

## C : ?? 

### Step2) 개별 모델 학습

In [ ]:
# 개별 모델들을 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [ ]:
# 학습되 ㄴ개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고
# 개별 모델의 정확도 측정

knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도: {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.9211
랜덤포레스트 정확도: 0.9649
결정트리 정확도: 0.9035
에이다부스트 정확도: 0.9561


### Step3) 개별 모델로부터 도출한 예측값을/ 칼럼 레벨로 옆에 붙여서/ 피처값으로 만들어// 
###    최종 메타 모델인 로지스틱 회귀에서 학습데이터로 다시 사용

In [ ]:
## 개별모델로부터 도출한 예측값을 피처 열로 만들어서 붙인다음
## 그걸 최종메타 모델의 학습데이터로 사용

# 예측결과는 1차원 형태의 ndarray이므로, 예측결과를 행형태로 붙인뒤
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)
# transpose()를 이용해 행과 열 위치를 바꾼 ndarray로 변환
pred = np.transpose(pred)
print(pred.shape)



(4, 114)
(114, 4)


In [ ]:
pred[1:100]

### Step4) 최종메타 모델인 로지스틱 회귀를 학습하고 예측 정확도를 측정

In [ ]:
# 개별모델에서 도출한 pred를
# 최종메타 모델에서는 X_test으로 활용 
lr_final.fit(pred, y_test) ## 일반적으로 fit(X_train, y_train)
final_pred = lr_final.predict(pred) ## predict(X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, final_pred))) ## acc_scr(y_test, pred)

## 특이한 점은 
## 일반적인 경우 학습할때 train dataset으로, 예측할때 test dataset로 하지만
## 최종메타 모델에서 학습 및 예측할 때 모두 개별모델에서 도출한 pred(test dataset)로 가지고 수행
## 따라서 학습할때는 train으로 안하고 test로 해서 과적합문제가 발생할 수 있음

최종 메타 모델의 예측 정확도: 0.9649


* 결과 분석  
1) 개별모델의 정확도보다 최종메타 모델의 정확도가 같거나 향상됨 

## 3. CV세트기반의 스태킹

* 과적합을 개선하기 위해//  
* 최종메타 모델을 위한 데이터 세트를 만들 때/ 교차검증 기반으로 예측된/ 결과 데이터세트를 이용 

* 이를 개선하기 위해
* 개별모델들이 각각 교차 검증으로/ 메타모델을 위한 학습용 스태킹 데이터 생성과/ 예측을 위한 테스트용 스태킹 데이터 생성을 한 뒤
* 이를 기반으로 메타모델이 학습과 예측을 수행한다. 


* 2단계의 스텝으로 구분  

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False) #, random_state=0) 이거하면 오류뜸
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    # 학습
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 테스트
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [ ]:
# 각 모델별 학습/테스트 데이터 합치기
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)


원본 학습 피처 데이터 Shape: (455, 30) 원본 테스트 피처 Shape: (114, 30)
스태킹 학습 피처 데이터 Shape: (455, 4) 스태킹 테스트 피처 데이터 Shape: (114, 4)


In [ ]:
# 최종 메타 모델 돌리기
lr_final.fit(Stack_final_X_train, y_train) # 최종 메타 모델 돌릴 때, 원본 학습 라벨 y_train 가져야 써야함
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9737
